In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

In [ ]:
# ---------- 1️⃣ HISTORICAL WEATHER DATA (3 months) ----------
def fetch_historical_weather(lat=51.5072, lon=-0.1276):
    """Fetch past 92 days of weather data using past_days parameter."""
    url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}"
        f"&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m,"
        f"cloudcover,shortwave_radiation"
        f"&past_days=92"
    )

    print("Fetching historical weather data (past 92 days)...")
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    df_weather = pd.DataFrame(
        {
            "datetime": data["hourly"]["time"],
            "temperature_C": data["hourly"]["temperature_2m"],
            "humidity_%": data["hourly"]["relative_humidity_2m"],
            "wind_speed_mps": data["hourly"]["wind_speed_10m"],
            "cloud_cover_%": data["hourly"]["cloudcover"],
            "solar_radiation_Wm2": data["hourly"]["shortwave_radiation"],
        }
    )
    df_weather["datetime"] = pd.to_datetime(df_weather["datetime"])
    print(f"Weather data: {len(df_weather)} records")
    return df_weather

In [ ]:
# ---------- 2️⃣ HISTORICAL AIR QUALITY DATA (3 months) ----------
def fetch_historical_air_quality(lat=51.5072, lon=-0.1276):
    """Fetch past 92 days of air quality data using past_days parameter."""
    url = (
        f"https://air-quality-api.open-meteo.com/v1/air-quality?"
        f"latitude={lat}&longitude={lon}"
        f"&hourly=pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,"
        f"sulphur_dioxide,ozone,us_aqi"
        f"&past_days=92"
    )

    print("Fetching historical air quality data (past 92 days)...")
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    df_aqi = pd.DataFrame(
        {
            "datetime": pd.to_datetime(data["hourly"]["time"]),
            "pm10": data["hourly"]["pm10"],
            "pm2_5": data["hourly"]["pm2_5"],
            "co": data["hourly"]["carbon_monoxide"],
            "no2": data["hourly"]["nitrogen_dioxide"],
            "so2": data["hourly"]["sulphur_dioxide"],
            "o3": data["hourly"]["ozone"],
            "aqi_us": data["hourly"]["us_aqi"],
        }
    )
    print(f"Air quality data: {len(df_aqi)} records")
    return df_aqi

In [ ]:
def fetch_historical_carbon_intensity():
    """Fetch past 3 months of UK carbon intensity data."""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=92)

    all_data = []
    current_date = start_date

    print("Fetching historical carbon intensity (past 92 days)...")

    while current_date <= end_date:
        month_end = min(current_date + timedelta(days=30), end_date)
        date_str = current_date.strftime("%Y-%m-%d")
        end_str = month_end.strftime("%Y-%m-%d")

        url = f"https://api.carbonintensity.org.uk/intensity/{date_str}/{end_str}"

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json().get("data", [])

            for r in data:
                all_data.append(
                    {
                        "datetime": r.get("from"),
                        "carbon_intensity_actual": r.get("intensity", {}).get("actual"),
                        "carbon_intensity_forecast": r.get("intensity", {}).get(
                            "forecast"
                        ),
                        "carbon_index": r.get("intensity", {}).get("index"),
                    }
                )

            time.sleep(0.5)
        except Exception as e:
            print(f"⚠️ Error fetching {url}: {e}")
            continue

        current_date = month_end + timedelta(days=1)

    df_carbon = pd.DataFrame(all_data)
    df_carbon["datetime"] = pd.to_datetime(df_carbon["datetime"])
    print(f"Carbon intensity data: {len(df_carbon)} records")
    return df_carbon

In [ ]:
def fetch_historical_generation_mix():
    """Fetch generation mix data (sampled every 12 hours for past 92 days)."""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=92)

    all_data = []
    sample_dates = pd.date_range(start=start_date, end=end_date, freq="12h")

    print("Fetching historical generation mix (sampled)...")

    for i, dt in enumerate(sample_dates, 1):
        print(
            f"  🔄 Fetching generation mix {i}/{len(sample_dates)} for {dt.strftime('%Y-%m-%d %H:%M')}..."
        )
        date_str = dt.strftime("%Y-%m-%dT%H:%MZ")
        url = f"https://api.carbonintensity.org.uk/generation/{date_str}/pt24h"

        try:
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json().get("data", [])

                for record in data:
                    timestamp = record.get("from")
                    gen_mix = record.get("generationmix", [])

                    mix_dict = {"datetime": timestamp}
                    for item in gen_mix:
                        fuel = item["fuel"].lower().replace(" ", "_")
                        mix_dict[f"uk_gen_{fuel}_%"] = item["perc"]

                    all_data.append(mix_dict)

            time.sleep(0.3)
        except Exception as e:
            print(f"⚠️ Skipped {dt.strftime('%Y-%m-%d %H:%M')} — {e}")
            continue

    df_gen = pd.DataFrame(all_data)
    if not df_gen.empty:
        df_gen["datetime"] = pd.to_datetime(df_gen["datetime"])
    print(f"✅ Generation mix data fetched: {len(df_gen)} records")
    return df_gen

In [ ]:
def fetch_historical_octopus_prices():
    """Fetch past 3 months of Octopus Energy prices."""
    products_url = "https://api.octopus.energy/v1/products/"
    response = requests.get(products_url)
    response.raise_for_status()

    products_data = response.json()
    agile_products = [
        p for p in products_data.get("results", []) if "AGILE" in p["code"]
    ]

    if not agile_products:
        raise ValueError("No Agile tariffs found")

    latest_agile = agile_products[0]
    product_code = latest_agile["code"]

    tariff_code = None
    for link in latest_agile.get("links", []):
        if "electricity-tariffs" in link.get("href", ""):
            tariff_code = link["href"].split("/")[-2]
            break
    if not tariff_code:
        tariff_code = f"E-1R-{product_code}-A"

    end_date = datetime.now()
    start_date = end_date - timedelta(days=92)

    period_from = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    period_to = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = (
        f"https://api.octopus.energy/v1/products/{product_code}/"
        f"electricity-tariffs/{tariff_code}/standard-unit-rates/"
        f"?period_from={period_from}&period_to={period_to}"
        f"&page_size=1500"
    )

    print("Fetching historical Octopus prices (past 92 days)...")

    all_prices = []
    while url:
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            results = data.get("results", [])
            all_prices.extend(results)

            url = data.get("next")
            if url:
                time.sleep(0.3)
        except Exception as e:
            print(f"⚠️ Error fetching Octopus prices: {e}")
            break

    df_prices = pd.DataFrame(all_prices)
    df_prices["datetime"] = pd.to_datetime(df_prices["valid_from"])
    df_prices["retail_price_£_per_kWh"] = df_prices["value_inc_vat"] / 100
    df_prices = df_prices[["datetime", "retail_price_£_per_kWh"]]
    print(f"Electricity prices: {len(df_prices)} records")
    return df_prices

In [ ]:
# ---------- 6️⃣ MERGE ALL SOURCES ----------
def merge_all_sources(weather_df, aqi_df, carbon_df, carbon_gen_df, prices_df):
    """Merge all data sources on datetime (normalize to UTC)."""
    dfs = [weather_df, aqi_df, carbon_df, prices_df]

    for df in dfs:
        if df["datetime"].dt.tz is None:
            df["datetime"] = df["datetime"].dt.tz_localize("UTC")
        else:
            df["datetime"] = df["datetime"].dt.tz_convert("UTC")

    merged = weather_df.merge(aqi_df, on="datetime", how="outer")
    merged = merged.merge(carbon_df, on="datetime", how="outer")
    merged = merged.merge(prices_df, on="datetime", how="outer")

    if not carbon_gen_df.empty:
        if carbon_gen_df["datetime"].dt.tz is None:
            carbon_gen_df["datetime"] = carbon_gen_df["datetime"].dt.tz_localize("UTC")
        else:
            carbon_gen_df["datetime"] = carbon_gen_df["datetime"].dt.tz_convert("UTC")

        merged = merged.merge(carbon_gen_df, on="datetime", how="left")

        for col in carbon_gen_df.columns:
            if col != "datetime" and col in merged.columns:
                merged[col] = merged[col].fillna(method="ffill").fillna(method="bfill")

    merged = merged.sort_values("datetime").reset_index(drop=True)
    return merged

In [ ]:
# ---------- 7️⃣ MAIN COLLECTION ----------
def collect_historical_data(save_dir="data", file_name="uk_energy_data.csv"):
    """
    Collect 3 months of historical data.
    Saves to the same file that daily collection will append to.
    """
    try:
        weather_df = fetch_historical_weather()
        aqi_df = fetch_historical_air_quality()
        carbon_df = fetch_historical_carbon_intensity()
        carbon_gen_df = fetch_historical_generation_mix()
        prices_df = fetch_historical_octopus_prices()

        print("\nMerging all datasets...")
        merged_df = merge_all_sources(
            weather_df, aqi_df, carbon_df, carbon_gen_df, prices_df
        )

        # --- 🧹 Clean up duplicate & half-hour entries ---
        merged_df["datetime"] = pd.to_datetime(merged_df["datetime"])

        # Round timestamps down to the nearest hour (drops 30-min marks)
        merged_df["datetime"] = merged_df["datetime"].dt.floor("H")

        # Drop duplicate timestamps (keep first)
        merged_df = merged_df.drop_duplicates(subset=["datetime"], keep="first")

        # Sort by datetime and reset index
        merged_df = merged_df.sort_values("datetime").reset_index(drop=True)

        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, file_name)

        merged_df.to_csv(save_path, index=False)
        print(f"\nHistorical data saved to: {save_path}")
        print(f"Total records: {len(merged_df)}")
        print(
            f"Date range: {merged_df['datetime'].min()} to {merged_df['datetime'].max()}"
        )

        return merged_df

    except Exception as e:
        print(f"Error during historical data collection: {e}")
        raise

In [ ]:
if __name__ == "__main__":
    df = collect_historical_data()